In [2]:
from pywinauto.application import Application
import pywinauto
from pywinauto import *
from time import sleep
from tqdm import tqdm
from pywinauto.timings import wait_until, wait_until_passes

In [11]:
def force_valid():
    app = Application(backend='uia').connect(best_match='Kofax Transformation Modules - Validation')
    app_dlg = app.window(best_match='Kofax Transformation Modules - Validation')

    invalid_docs = int(app_dlg.StatusBar.children()[-1].window_text().replace(' ', '').split(':')[-1])
    doc_ke = 0
    
    while(doc_ke<(invalid_docs)):
        print('# doc ke: {}'.format(doc_ke+1))

        # check unclassified doc
        unclass_doc = app_dlg.Pane.children()[0].children()[0].children()[0].children(title='Document is unclassified, please select a class first.')
        if(len(unclass_doc)>0):
            print('reclassify')
            combo = app_dlg.child_window(auto_id="PanelClassCombo")
            combo.children()[0].type_keys("%{DOWN}")
            combo.children()[0].expand()
            combo.children()[0].children()[0].children()[10].select().type_keys('{ENTER}')
            reclass_dlg_yes = app_dlg.descendants(title='Yes', control_type="Button")
            reclass_dlg_yes[0].click()

            app = Application(backend='uia').connect(best_match='Kofax Transformation Modules - Validation')
            app_dlg = app.window(best_match='Kofax Transformation Modules - Validation')

            save_dlg = app_dlg.descendants(title="Do you want to save the current document?")
            print(save_dlg)
            if(len(save_dlg)>0):
                # suspend batch
                doc_ke+=1
                print("# -- save doc")
                save_doc_yes = app_dlg.descendants(title="Yes", control_type="Button")
                save_doc_yes[0].click()
                continue

            
        # check invalid fields
        invalid_fields = int(app_dlg.child_window(auto_id='lblFieldStatus').window_text().split(' ')[3])
        if(invalid_fields==0):
            app_dlg.type_keys("{ENTER}")
            save_dlg = app_dlg.descendants(title="Do you want to save the current document?")
            if(len(save_dlg)>0):
                # suspend batch
                doc_ke+=1
                print("# -- save doc")
                save_doc_yes = app_dlg.descendants(title="Yes", control_type="Button")
                save_doc_yes[0].click()
                continue
        else:
            # force all fields can custom setting keyboard
            app_dlg.type_keys('^+F')
            force_all_yes = app_dlg.descendants(title='Yes', control_type="Button")
            force_all_yes[0].click()

        save_dlg = app_dlg.descendants(title="Do you want to save the current document?")
        if(len(save_dlg)>0):
            # suspend batch
            doc_ke+=1
            print("# -- save doc")
            save_doc_yes = app_dlg.descendants(title="Yes", control_type="Button")
            save_doc_yes[0].click()

    close_ok = app_dlg.descendants(title="OK")
    if(len(close_ok)>0):
        close_ok[0].click()
    

# JANGAN DIRUN DULU (INI PILIHAN TERAKHIR)

In [ ]:
# 16OP
# BATCH

jml_batch = 1
batch_done = 0
while(batch_done<jml_batch):
    try:
        print('batch: {}'.format(batch_done+1))
        force_valid()
        batch_done+=1
    except:
        pass